# Download and Inspect the Collection

The dataset was created from the Chronicling America collection — over 21 million digitized newspaper pages (1756–1963) curated by the Library of Congress and NEH. They used 39,330 pages (1800–1920), representing 53 US states, to ensure wide geographic and temporal coverage.

Source: https://dl.acm.org/doi/pdf/10.1145/3626772.3657891

GitHub: https://github.com/DataScienceUIBK/ChroniclingAmericaQA?tab=readme-ov-file

In [1]:
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/test.json?download=true" -o test.json
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/train.json?download=true" -o train.json
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/dev.json?download=true" -o validation.json

import json

files = ["train.json", "validation.json", "test.json"]

for path in files:
    print(f"\n===== {path} =====")
    try:
        with open(path, "r", encoding="utf-8") as f:
            # Read a few hundred characters to see what kind of JSON it is
            head = f.read(500)
            print("Preview of first 500 characters:\n")
            print(head[:500])
        # Try to load only part of the file
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            print(f"\nLoaded {len(data)} items (list).")
            print("Dictionary keys:", list(data[0].keys()))
            print(json.dumps(data[0], indent=2)[:600])
        elif isinstance(data, dict):
            print("\nTop-level is a dictionary. Keys:", list(data.keys()))
            for k, v in data.items():
                if isinstance(v, list):
                    print(f"Key '{k}' contains a list of {len(v)} items.")
                    if v:
                        print("First item keys:", list(v[0].keys()))
                        print(json.dumps(v[0], indent=2)[:600])
                        break
        else:
            print(f"Unexpected top-level type: {type(data)}")
    except Exception as e:
        print(f"Could not parse {path} as JSON: {e}")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1350  100  1350    0     0   6376      0 --:--:-- --:--:-- --:--:--  6459

  3 71.5M    3 2683k    0     0  5702k      0  0:00:12 --:--:--  0:00:12 5702k
 26 71.5M   26 19.1M    0     0  13.0M      0  0:00:05  0:00:01  0:00:04 16.5M
 50 71.5M   50 36.4M    0     0  14.7M      0  0:00:04  0:00:02  0:00:02 16.9M
 77 71.5M   77 55.1M    0     0  15.8M      0  0:00:04  0:00:03  0:00:01 17.5M
100 71.5M  100 71.5M    0     0  16.4M      0  0:00:04  0:00:04 --:--:-- 17.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1350  100  1350    0     0   7059      0 --:


===== train.json =====
Preview of first 500 characters:

[
    {
        "query_id": "train_1",
        "question": "Who is the author of the book, \"Horrors of Slavery, or the American Turf in Tripoli\"?",
        "answer": "WILLIAM RAY",
        "org_answer": "WILLIAM RAY",
        "para_id": "New_Hampshire_18070804_1",
        "context": "Aiscellaneous Repository. From the Albany Register, WAR, OR A PROSPECT OF IT, From recent instances of British Outrage. BY: WILLIAM RAY, Author of the contemplated publication, entitled, \u201cHorrors of Slavery, 

Loaded 439302 items (list).
Dictionary keys: ['query_id', 'question', 'answer', 'org_answer', 'para_id', 'context', 'raw_ocr', 'publication_date', 'trans_que', 'trans_ans', 'url']
{
  "query_id": "train_1",
  "question": "Who is the author of the book, \"Horrors of Slavery, or the American Turf in Tripoli\"?",
  "answer": "WILLIAM RAY",
  "org_answer": "WILLIAM RAY",
  "para_id": "New_Hampshire_18070804_1",
  "context": "Aiscellaneous R

# Create the Document Collection

To do that, we create a new json file that contains the 'para_id', 'context', 'raw_ocr', 'publication_date' keys, for all para_id in the collection.

para_id: is the id of a paragraph of a news paper page.

In [3]:
import json
import os

inputs = ["train.json", "validation.json", "test.json"]
output = "document_collection.json"

def load_list_or_empty(path):
    if not os.path.exists(path) or os.path.getsize(path) == 0:
        print(f"Skipping {path} because it is missing or empty")
        return []
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            return data
        print(f"Skipping {path} because it is not a list at the top level")
        return []
    except json.JSONDecodeError:
        print(f"Skipping {path} because it is not valid JSON")
        return []

def project(recs):
    out = []
    for r in recs:
        out.append({
            "para_id": r.get("para_id", ""),
            "context": r.get("context", ""),
            "raw_ocr": r.get("raw_ocr", ""),
            "publication_date": r.get("publication_date", "")
        })
    return out

all_recs = []
for p in inputs:
    recs = load_list_or_empty(p)
    print(f"Loaded {len(recs)} records from {p}")
    all_recs.extend(project(recs))

# deduplicate by para_id keeping the first one seen
uniq = {}
for rec in all_recs:
    pid = rec.get("para_id", "")
    if pid and pid not in uniq:
        uniq[pid] = rec

result = list(uniq.values())

with open(output, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print(f"Wrote {len(result)} records to {output}")
print(json.dumps(result[:3], indent=2))

Loaded 439302 records from train.json
Loaded 24111 records from validation.json
Loaded 24084 records from test.json
Wrote 131921 records to document_collection.json
[
  {
    "para_id": "New_Hampshire_18070804_1",
    "context": "Aiscellaneous Repository. From the Albany Register, WAR, OR A PROSPECT OF IT, From recent instances of British Outrage. BY: WILLIAM RAY, Author of the contemplated publication, entitled, \u201cHorrors of Slavery, or the American Turf in Tripoli,\u201d VOTARIES of Freedom, arm! The British Lion roars! Legions of Valor, take th\u2019 alarm\u2014; Rash, rush to guard our shores! Behold the horrid deed\u2014 Your brethren gasping lie! Beneath a tyrant\u2019s hand they bleed\u2014 They groan\u2014they faint\u2014they die. Veterans of seventy-six, Awake the slumbering sword;\u2014 Hearts of your murderous foes transfix\u2014 'Tis vengeance gives the word. Remember Lexington, And Bunker\u2019s tragic hill; \u201cThe same who spilt your blood thereon, Your blood again

## You should check that the collection you have matches that of the paper!

# Create the Test Queries Data Structure

We keep the first 10.000 queries due to memory errors in the free colab version.

To be comparable, please keep the top 10.000 queries for evaluation.

In [4]:
import json
import re
import unicodedata
import string

input_file = "test.json"
output_file = "test_queries.json"

# Load the data
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

def clean_question(text):
    if not isinstance(text, str):
        return ""
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(rf"[{re.escape(string.punctuation)}]", " ", text)  # remove punctuation
    text = re.sub(r"\s+", " ", text)  # collapse multiple spaces
    return text.strip()

# Extract and clean
queries = [
    {
        "query_id": item.get("query_id", ""),
        "question": clean_question(item.get("question", "")),
    }
    for item in data
]

# Sort by query_id (assuming numeric)
queries = sorted(queries, key=lambda x: int(x["query_id"]) if str(x["query_id"]).isdigit() else x["query_id"])

# Keep only the first 10,000
queries = queries[:10000]

# Save new JSON
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(queries, f, ensure_ascii=False, indent=2)

print(f"Saved {len(queries)} entries to {output_file}")
print(json.dumps(queries[:3], indent=2))

Saved 10000 entries to test_queries.json
[
  {
    "query_id": "test_1",
    "question": "How many lots did Thomas Peirce have"
  },
  {
    "query_id": "test_10",
    "question": "Who gave Hamilton the substance of what he had proposed on the part of General Hamilton"
  },
  {
    "query_id": "test_100",
    "question": "Who informs his FRIENDS and the PUBLIC that he has taken that justly celebrated INN in this city"
  }
]


# Create the Qrels for the test set

In [5]:
input_file = "test.json"
qrels_file = "test_qrels.json"
answers_file = "test_query_answers.json"

# Load the data
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Build the qrels file: query_id, iteration=0, para_id, relevance=1
qrels = [
    {
        "query_id": item.get("query_id", ""),
        "iteration": 0,
        "para_id": item.get("para_id", ""),
        "relevance": 1
    }
    for item in data
]

# Build the query_answers file: same plus answer and org_answer
query_answers = [
    {
        "query_id": item.get("query_id", ""),
        "iteration": 0,
        "para_id": item.get("para_id", ""),
        "relevance": 1,
        "answer": item.get("answer", ""),
        "org_answer": item.get("org_answer", "")
    }
    for item in data
]

# Save both files
with open(qrels_file, "w", encoding="utf-8") as f:
    json.dump(qrels, f, ensure_ascii=False, indent=2)

with open(answers_file, "w", encoding="utf-8") as f:
    json.dump(query_answers, f, ensure_ascii=False, indent=2)

print(f"Saved {len(qrels)} entries to {qrels_file}")
print(f"Saved {len(query_answers)} entries to {answers_file}")
print("Sample qrels entry:", qrels[0])
print("Sample query_answers entry:", query_answers[0])

Saved 24084 entries to test_qrels.json
Saved 24084 entries to test_query_answers.json
Sample qrels entry: {'query_id': 'test_1', 'iteration': 0, 'para_id': 'New_Hampshire_18030125_16', 'relevance': 1}
Sample query_answers entry: {'query_id': 'test_1', 'iteration': 0, 'para_id': 'New_Hampshire_18030125_16', 'relevance': 1, 'answer': '183', 'org_answer': '183'}


# Retrieval - Good Luck!

# Get the Index of the collection and store the document as a dataframe
inverted index


In [6]:
import os
import pyterrier as pt
import json
import pandas as pd 
import shutil

#path index folder 
folder_name = "Index"
index_file_name = "terrier_inverted_index"
index_path = os.path.abspath(os.path.join(folder_name, index_file_name))

#init pyTer
if not pt.java.started():
    pt.java.init()

#make a data frame from all the documents
#build Data frame and clean the row context field 
def  get_dataFrame():
    json_path = 'document_collection.json'
    parquet_path = 'documents.parquet'
    if os.path.exists(parquet_path):
        df_documents = pd.read_parquet(parquet_path)
    else:
        with open('document_collection.json', 'r', encoding='utf-8') as f:
            raw_data = json.load(f)
        df_documents = pd.DataFrame(raw_data)
        df_documents = df_documents.rename(columns={"para_id": "docno", "context": "text"})[["docno", "text", "publication_date"]]
        df_documents = df_documents.set_index('docno')
        df_documents.to_parquet(parquet_path)
    return df_documents
    

def get_index(df):
    #check if the index exsists
    properties_file = os.path.join(index_path, "data.properties")
    if os.path.exists(properties_file):
        return pt.IndexFactory.of(index_path)
    print("Index is not found, creating a new Index")
    if not os.path.exists(index_path):
        os.makedirs(index_path, exist_ok=True)
   
    
    # Build the index
    # Build the index using the updated IterDictIndexer signature
    # Key parameters now are: meta, text_attrs, meta_reverse, pretokenised, fields, threads
    indexer = pt.IterDictIndexer(
        index_path,
        meta={"docno": 40},            # store docno as metadata (up to 20 characters)
        text_attrs=["text"],           # which field(s) contain the text
        meta_reverse=["docno"],        # enable reverse lookup on docno
        pretokenised=False,
        fields=False,
        threads=1,
    )
    index_ref = indexer.index(df.to_dict(orient="records"))
    return pt.IndexFactory.of(index_ref)
    
df_documents = get_dataFrame()
index = get_index(df_documents)

# Print a simple summary
print("Index location:", index_path)
print("Indexed documents:", index.getCollectionStatistics().getNumberOfDocuments())



Java started and loaded: pyterrier.java.colab, pyterrier.java, pyterrier.java.24, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


Index location: C:\Users\user\Desktop\school\3\Bicoca\InformationRetrival\Project\Index\terrier_inverted_index
Indexed documents: 131921


# stats of the indexing


In [7]:
# Retrieve collection statistics
stats = index.getCollectionStatistics()

print("Terrier Collection Statistics")
print("--------------------------------")
print(f"Indexed documents:        {stats.getNumberOfDocuments()}")
print(f"Unique terms (vocabulary): {stats.getNumberOfUniqueTerms()}")
print(f"Total tokens:             {stats.getNumberOfTokens()}")
print(f"Average document length:  {stats.getAverageDocumentLength():.2f}")

Terrier Collection Statistics
--------------------------------
Indexed documents:        131921
Unique terms (vocabulary): 236646
Total tokens:             15575099
Average document length:  118.06


In [8]:
# First stage retriver with BM25 
bm25 = pt.terrier.Retriever(index, wmodel="BM25")


In [9]:
#testing the engine 
query = "Europe"
results = bm25.search(query)

#show first 10 resaults
print(results.head(10))


  qid   docid                     docno  rank      score   query
0   1  122024      New_York_19190610_27     0  10.233243  Europe
1   1   61883         Texas_18771215_13     1   9.759719  Europe
2   1    9637      Maryland_18401110_20     2   9.730148  Europe
3   1  110750      Nebraska_19020225_26     3   9.715430  Europe
4   1   36539  Rhode_Island_18511030_25     4   9.642500  Europe
5   1   47808      Illinois_18680907_40     5   9.578485  Europe
6   1  100207         Kansas_18970902_5     6   9.570657  Europe
7   1   65964     Tennessee_18740915_14     7   9.556417  Europe
8   1   29371        Hawaii_18520522_33     8   9.509314  Europe
9   1   38017     Tennessee_18590726_13     9   9.499877  Europe


In [10]:
#gettting the text of the document by its "docno"
def get_text_by_docno(docno):
    return df_documents.loc[docno]['text']


# hist word vectors embedding
using hist word embadding to retrive the closest term to the query term for each decade from 1800 to 1920. 


In [11]:
#changing the root directory to hist_words
import sys 
query_word = "gay"
path = os.getcwd()
home_dir = path.split("\\")[-1]
if( home_dir == "histwords_master"): 
    os.chdir("..") # changing root directory to the project root folder
project_home = os.path.abspath("histwords_master")
if project_home not in sys.path:
    sys.path.insert(0, project_home)
home_dir = path.split("\\")[-1]
print(home_dir)
if( home_dir != "histwords_master"): 
    os.chdir(project_home)
print(f"Working directory set to: {os.getcwd()}")
try:
    from representations.sequentialembedding import SequentialEmbedding
    
    fiction_embeddings = SequentialEmbedding.load("embeddings/sgns", list(range(1810, 2000, 10)))
    
    similar_words  =fiction_embeddings.get_seq_closest(query_word,1810, 110 )
    print(similar_words)
except:
    print("query expension failed")
finally:
    path = os.getcwd()
    home_dir = path.split("\\")[-1]
    if( home_dir == "histwords_master"): 
        os.chdir("..") # changing root directory to the project root folder 


Project
Working directory set to: C:\Users\user\Desktop\school\3\Bicoca\InformationRetrival\Project\histwords_master
['gay', 'lovely', 'cheerful', 'flowers', 'merry', 'brilliant', 'graceful', 'bright', 'song', 'soft']


In [12]:
path = os.getcwd()
home_dir = path.split("\\")[-1]
if( home_dir == "histwords_master"): 
    os.chdir("..") # changing root directory to the original    


# query with bm 25 foreach word in the resault of the histWord similar words


In [13]:

results = bm25.search(query_word)
maximum_length = 100
for word in similar_words:
    temp_bm25_resaults =  bm25.search(word).head(100)
    results = pd.concat([results,temp_bm25_resaults] , ignore_index= True).drop_duplicates(subset='docid', keep='first')
    results = results.sort_values(by = "score", ascending=False)
    results = results.head(100)
results['rank'] = range(len(results))

results

,qid,docid,docno,rank,score,query
0,1,110609,Michigan_19091204_23,0,16.008571,merry
1,1,88533,North_Dakota_18811230_27,1,15.660783,merry
2,1,26305,Kansas_18590129_1,2,14.563645,merry
3,1,127730,Iowa_18490201_21,3,14.112891,gay
4,1,103992,North_Dakota_19001227_10,4,13.606007,merry
...,...,...,...,...,...,...
89,1,43913,West_Virginia_18700129_32,95,11.145719,bright
90,1,64379,Vermont_18780816_17,96,11.139544,flowers
91,1,44290,District_of_Columbia_18610222_4,97,11.130769,merry
92,1,117639,Utah_19110219_4,98,11.118203,song
